In [2]:
from simsopt.field import load_coils_from_makegrid_file
coils = load_coils_from_makegrid_file('LHD_data/lhd.coils.txt')

TypeError: load_coils_from_makegrid_file() missing 1 required positional argument: 'order'

In [ ]:
for coil in coils:
    coil.plot(engine='mayavi', show=False, close=True)